In [1]:
import functools

import pandas as pd

import ast

pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

In [2]:
DIR = 'data/20241230_173555'

df_all = pd.read_csv(f'{DIR}/df_all.csv', low_memory=False)
df_all

,Unnamed: 0,filename,dir_map,basename_scenario,i_map,position,passhum,slowness,forcing,filename_screenshot,are_bridges,configuration,isCanPassFirstHum,isCanPassFirstAut,reroutingsAtParked,reroutingsAtSlow,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s),Scenario,Current datetime,Time passed (real),Time passed (sim.),isCanPassFirstActive,isRacingThroughCrossroadAllowed,probabilitySlowingDownForHuman,Human V0,Collision events,Vehicle size (m),v_current,v_max,can pass first,violation of priorities,moving slowly,improper parking,cautious mode,reroutings at parked / slow,moving backwards,change of priorities,stops,violations,near misses,collisions,"traveled total, m",no. missions,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions,Linearization A,Linearization B,Linearization C,Linearization D1,Linearization D2,Linearization D3,Linearization D0,Linearization A (non-normalized),Linearization B (non-normalized),Linearization C (non-normalized),Linearization D1 (non-normalized),Linearization D2 (non-normalized),Linearization D3 (non-normalized),Linearization D0 (non-normalized),Linearization D (non-normalized),seconds_blocked,is_blocked
0,4,map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json,2024-11-28_13:17:39_with_bridges,scenario1-1,1,1,False,baseline,baseline,../map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/screenshots/scenario1-1.png,True,"map 1, with bridges, pos.var. 1",False,False,NaN,NaN,20241230_163609,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no",0,HumanDrivenVehicle,778.12,5,4007.4,24,0,0,75,0,0,775.3,134.8,1799.9,0.3,5.6,2.2,"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no",2024-12-30 16:38:12,0:02:03,0:29:59 (x14.6),"hum=false, aut=false",True,0.0,0 forcing events,"0 minor, 0 major","[length=10.0, width=10.0, safe distance: front=5.0, back=5.0, left=1.0, right=1.0]",1.5,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,4006.8,5,NaN,"(172.1, 148.4)",122.2,121.0,777,124,101.4,0.0,DRIVING,"2: [778, 777]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001318, 0.001...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305867, 0.305...",NaN,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [3]:
sorted(df_all.columns, key=lambda s: s.lower())

['are_bridges',
 'Average speed (m/s)',
 'basename_scenario',
 'blocked',
 'can pass first',
 'cautious mode',
 'change of priorities',
 'Collision events',
 'collisions',
 'configuration',
 'CP (index)',
 'Current datetime',
 'Cycle distance (m)',
 'Date',
 'dir_map',
 'distance ToCP, m',
 'filename',
 'filename_screenshot',
 'forcing',
 'future missions',
 'Human V0',
 'i_map',
 'improper parking',
 'is_blocked',
 'isCanPassFirstActive',
 'isCanPassFirstAut',
 'isCanPassFirstHum',
 'isRacingThroughCrossroadAllowed',
 'Linearization A',
 'Linearization A (non-normalized)',
 'Linearization B',
 'Linearization B (non-normalized)',
 'Linearization C',
 'Linearization C (non-normalized)',
 'Linearization D (non-normalized)',
 'Linearization D0',
 'Linearization D0 (non-normalized)',
 'Linearization D1',
 'Linearization D1 (non-normalized)',
 'Linearization D2',
 'Linearization D2 (non-normalized)',
 'Linearization D3',
 'Linearization D3 (non-normalized)',
 'Maximum acceleration (m/s^2)',

In [4]:
RESOLUTION = 1  # m/step (m/pixel)


def same_value(series):
    is_ok = series.nunique(dropna=False) == 1
    if not is_ok:
        print(series)
        raise ValueError('not same values')
    return series.iloc[0]


def id2name(vehicle_id):
    return f"{"MV" if vehicle_id == 0 else "AV"} (V{vehicle_id})"


def process_df_all(df_all):
    df_in = df_all[~df_all['passhum']]
    
    # Dictionary to map Map IDs to the number of OPs
    map_to_ops = {
        1: 2, 6: 2, 10: 2,  # Maps with 2 OPs
        2: 1, 3: 1, 4: 1, 5: 1, 7: 1, 8: 1, 9: 1,  # Maps with 1 OP
    }
    
    rows = []
    for name, group in df_in.groupby('Scenario ID'):
#        group_avs = group[group['Vehicle ID'] != 0]
        
        map_id = same_value(group['i_map'])

        is_forcing = same_value(group['forcing']) != 'baseline'
        is_slowness = same_value(group['slowness']) != 'baseline'
        
        is_with_bridges = same_value(group['dir_map']).endswith('_with_bridges')
        is_without_bridges = same_value(group['dir_map']).endswith('_without_bridges')
        assert is_with_bridges != is_without_bridges

        mission_lengths = pd.Series({
            vehicle_id: 
                len(linearization_to_tuple(
                    group[group['Vehicle ID'] == vehicle_id]['Linearization C (non-normalized)']
                )) * RESOLUTION
            for vehicle_id in range(4)
        })

        ids_mv = [0]
        ids_av = [1, 2, 3]
        
        row = {
            '': {
                'Scenario ID': name,
            },
            'Violation type': {
                'Priority violation': is_forcing and not is_slowness,
                'Speed violation': is_slowness and not is_forcing,
                'Both': is_forcing and is_slowness,
            },
            'Coordination strategy': {
                'Change of priorities': same_value(group['forcing']) == 'change of priorities',
                'Stops': same_value(group['forcing']) == 'stops',
                'Rerouting': same_value(group['slowness']) == 'with rerouting',
            },
            'Static map features': {
                'Map ID': map_id,
                'No. of OPs': map_to_ops.get(map_id, None),
                'Connectivity': 'high' if same_value(group['dir_map']).endswith('_with_bridges') else 'low',
                'Position': f"{map_id}-{same_value(group['position'])}",
            },
            'Output of simulation (planning)': {
                
                # CSD (Critical Section Density) score for MV (V0)
                'CSD score for MV (V0)': calculate_cs_score(group[group['Vehicle ID'] == 0]['Linearization C (non-normalized)']),

                # CSD score for each AV
                **{
                    f'CSD score for AV (V{vehicle_id})': calculate_cs_score(group[group['Vehicle ID'] == vehicle_id]['Linearization C (non-normalized)'])
                    for vehicle_id in range(1, 4)
                },
                # Mean CSD score for AVs (V1-V3)
                'Mean CSD score for AVs (V1-V3)': calculate_mean_cs_score(group, vehicle_ids=[1, 2, 3]),
                
                # Mission length for each AV (V1-V3)
                **{
                    f'Mission length for {id2name(vehicle_id)}': mission_lengths[vehicle_id]
                    for vehicle_id in range(4)
                },
                # Total and mean mission lengths for AVs (V1-V3)
                'Total Mission length for AVs (V1-V3)': mission_lengths[ids_av].sum(),
                'Mean Mission length for AVs (V1-V3)': mission_lengths[ids_av].mean(),
            },
            'Output of simulation (execution)': {
                # Completed missions
                **{
                    f'No. of completed missions for {id2name(vehicle_id)}': 
                        group[group['Vehicle ID'] == vehicle_id]['No. of completed missions'].sum()
                    for vehicle_id in range(4)
                },
                # Total completed missions for AVs (V1-V3)
                'Total No. of completed missions for AVs (V1-V3)': group[group['Vehicle ID'].isin([1, 2, 3])]['No. of completed missions'].sum(),

                # Collisions for MV (V0) first
                'No. of collisions for MV (V0)': group[group['Vehicle ID'] == 0]['No. of collisions'].sum(),

                # Collisions for each AV (V1-V3)
                **{
                    f'No. of collisions for AV (V{vehicle_id})': group[group['Vehicle ID'] == vehicle_id]['No. of collisions'].sum()
                    for vehicle_id in range(1, 4)
                },
                # Total collisions for AVs (V1-V3)
                'Total No. of collisions for AVs (V1-V3)': group[group['Vehicle ID'].isin([1, 2, 3])]['No. of collisions'].sum(),

                # Near misses for MV (V0) first
                'No. of near-misses for MV (V0)': group[group['Vehicle ID'] == 0]['No. of near-misses'].sum(),

                # Near misses for each AV (V1-V3)
                **{
                    f'No. of near-misses for AV (V{vehicle_id})': group[group['Vehicle ID'] == vehicle_id]['No. of near-misses'].sum()
                    for vehicle_id in range(1, 4)
                },
                # Total near misses for AVs (V1-V3)
                'Total No. of near-misses for AVs (V1-V3)': group[group['Vehicle ID'].isin([1, 2, 3])]['No. of near-misses'].sum(),
            },
        }
        rows.append(row)
        
    rows_flat = []
    for row in rows:
        row_flat = {}
        for group, values in row.items():
            for col, value in values.items():
                row_flat[group, col] = value
        rows_flat.append(row_flat)
        
    df_out = pd.DataFrame(rows_flat).round(3)
    df_out = df_out.reindex(columns=pd.MultiIndex.from_tuples(list(rows_flat[0])))
    
    df_out.to_csv(f'{DIR}/df_all_hadi.csv', index=False)
    return df_out


@functools.cache
def literal_eval(value):
    return ast.literal_eval(value)


def linearization_to_tuple(series):
    tup, = list(series)
    return literal_eval(tup)


def calculate_cs_score(series):
    """Converts string tuples to actual tuples and calculates the mean."""
    total_sum = 0
    count = 0
    for value in series:
        if isinstance(value, str):
            # Convert string to a tuple
            value = literal_eval(value)
        if isinstance(value, tuple):
            total_sum += sum(value)
            count += len(value)
        elif isinstance(value, (int, float)):
            total_sum += value
            count += 1
    return total_sum / count if count > 0 else None


def calculate_mean_cs_score(group, vehicle_ids):
    """Calculates the mean CS score across specified vehicle IDs."""
    cs_scores = [
        calculate_cs_score(group[group['Vehicle ID'] == vehicle_id]['Linearization C (non-normalized)'])
        for vehicle_id in vehicle_ids
    ]
    cs_scores = [score for score in cs_scores if score is not None]
    return sum(cs_scores) / len(cs_scores) if cs_scores else None


df_processed = process_df_all(df_all)
df_processed

\
                                                                                                                                                 Scenario ID   
0                       map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities   
1                                         map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no   
2                                      map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops   
3           map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing change of priorities   
4                             map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing no   
...                                                                                                                                                      ...   
1494                                   map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing no   
1495                                map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing stops   
1496  map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities   
1497                    map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing no   
1498                 map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops   

         Violation type                        Coordination strategy         \
     Priority violation Speed violation   Both  Change of priorities  Stops   
0                  True           False  False                  True  False   
1                 False           False  False                 False  False   
2                  True           False  False                 False   True   
3                 False           False   True                  True  False   
4                 False            True  False                 False  False   
...                 ...             ...    ...                   ...    ...   
1494              False           False  False                 False  False   
1495               True           False  False                 False   True   
1496              False           False   True                  True  False   
1497              False            True  False                 False  False   
1498              False           False   True                 False   True   

               Static map features                                   \
     Rerouting              Map ID No. of OPs Connectivity Position   
0        False                   1          2         high      1-1   
1        False                   1          2         high      1-1   
2        False                   1          2         high      1-1   
3         True                   1          2         high      1-1   
4         True                   1          2         high      1-1   
...        ...                 ...        ...          ...      ...   
1494     False                   9          1          low      9-9   
1495     False                   9          1          low      9-9   
1496     False                   9          1          low      9-9   
1497     False                   9          1          low      9-9   
1498     False                   9          1          low      9-9   

     Output of simulation (planning)                        \
               CSD score for MV (V0) CSD score for AV (V1)   
0